In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

import pickle

In [4]:
# read data from csv G:\.shortcut-targets-by-id\1CZZx9Bpz7hB-BhPFsqAL3fRVyxM1XfwF\AirBnB_project
df = pd.read_csv('G:\\.shortcut-targets-by-id\\1CZZx9Bpz7hB-BhPFsqAL3fRVyxM1XfwF\\AirBnB_project\\listings_comments.csv')

In [ ]:
# read data 
df = pd.read_csv('dataset\listings_comments.csv')

In [5]:
df.comments

0       ['Daniel is really cool. The place was nice an...
1       ['Nice apartment in a beautifull área of Amste...
2       ['Einfach große Klasse! Zentrale Lage, Einkauf...
3       ["This was by far the best Airbnb experience I...
4       ['Amazing house, perfect location and a dream ...
                              ...                        
6993                                                  NaN
6994                                                  NaN
6995                                                  NaN
6996                                                  NaN
6997                                                  NaN
Name: comments, Length: 6998, dtype: object

In [20]:
# load the model that is responsible for the text embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# create a device object
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
# move the model to the device
model.to(device)




# parallelize the model across multiple GPUs(if available)
# model = nn.DataParallel(model)

Device: cuda


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [21]:
# each row have multiple comments, so we need to get the embeddings for each comment and then average them
# to get the embedding for the row

def get_row_embeddings(df, column):
    # convert comments to strings
    comments = df[column].astype(str)
    # get the embeddings for each comment
    embeddings = model.encode(comments.values, show_progress_bar=True)
    # get the average of the embeddings
    row_embeddings = np.mean(embeddings, axis=0)
    return embeddings, row_embeddings



In [22]:

# get the embeddings for the comments
comments_embeddings, mean_emb = get_row_embeddings(df, 'comments')

Batches: 100%|██████████| 219/219 [00:51<00:00,  4.22it/s]


In [26]:
comments_embeddings.shape

(384,)

In [27]:
import pickle
with open('comments_embeddings.pkl', 'wb') as f:
    pickle.dump(comments_embeddings, f)

with open('mean_emb.pkl', 'wb') as f:
    pickle.dump(mean_emb, f)